In [1]:
%cd ~/cdv
!source env.sh

/home/nmiklaucic/cdv


/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'

import numpy as np
import jax.numpy as jnp
import jax.random as jr
import jax
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp

is_dark = False
theme, cs = rp.mpl_setup(is_dark)

In [3]:
from pathlib import Path
import pyrallis
from cdv.config import MainConfig
import orbax.checkpoint as ocp

from cdv.training_state import TrainingRun
from cdv.checkpointing import best_ckpt

# run_dir = Path('logs') / '03-17:45_918'
# run_dir = Path('logs') / 'e_form_no_downsample_564'
# run_dir = Path('logs') / 'e_form_equivariant_patch_235'
# run_dir = Path('logs') / '04-13-interrupted-919'
# run_dir = Path('logs') / '04-13-14_650'
run_dir = Path('logs') / 'mace_vae_rec_407'

with open(run_dir / 'config.toml') as conf_file:
    config = pyrallis.cfgparsing.load(MainConfig, conf_file)

model = config.build_vae()

ckpt = best_ckpt(run_dir)
# ckpt = jax.tree.map(lambda x: x if isinstance(x, (float, int)) else x.astype(jnp.bfloat16), ckpt)
model = model.bind(ckpt['state']['params'])

Configured `CheckpointManager` using deprecated legacy API. Please follow the instructions at                      
https://orbax.readthedocs.io/en/latest/api_refactor.html to migrate by May 1st, 2024.

Given directory is read only=/home/nmiklaucic/cdv/logs/mace_vae_rec_407/final_ckpt/ckpts

Restoring item from /home/nmiklaucic/cdv/logs/mace_vae_rec_407/final_ckpt/ckpts/41799.

Finished restoring checkpoint from /home/nmiklaucic/cdv/logs/mace_vae_rec_407/final_ckpt/ckpts/41799.

In [4]:
from cdv.dataset import load_file

batch = load_file(config, 0, pad=True)
batch


CrystalGraphs(
    nodes=NodeData(
        species=Array([28, 28, 28, 28, 70, 70, 70, 70, 71, 71, 71, 71, 71, 71, 71, 71, 71,
       71, 71, 71, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 63, 63,
       63, 63, 64, 64, 70, 70,  6,  6, 11, 11, 26, 26, 26, 26, 70, 70, 70,
       70, 70, 70, 70, 70, 70, 70, 70, 70, 31, 31, 31, 31, 31, 31, 70, 70,
       70, 70, 70, 70, 70, 70, 70, 70, 70, 71, 37, 37, 37, 37, 37, 37, 70,
       70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 71, 26, 26, 26, 26, 53, 53,
       53, 53, 70, 70, 70, 70, 70, 70, 70, 70, 37, 37, 37, 37, 71, 71, 71,
       71, 71, 71, 71, 71, 71, 71, 71, 71, 15, 15, 15, 15, 50, 50, 50, 50,
       63, 63, 63, 63, 63, 63, 63, 63,  5,  5, 31, 31, 36, 36, 70, 70, 70,
       70, 70, 70, 70, 70, 50, 50, 50, 50, 70, 70, 70, 70, 70, 70, 70, 70,
       70, 70, 14, 14, 14, 20, 20, 33, 65, 65, 65, 65, 65, 65, 65, 65,  2,
        2, 15, 48, 48, 48, 70, 70, 70, 70, 70, 70, 70, 70, 31, 31, 37, 37,
       70, 70, 70, 70, 70, 70, 70, 70, 18, 18, 5

In [5]:
from cdv.utils import debug_structure


debug_structure(batch);

arg0 >>> CrystalGraphs
├── nodes
│   ├── species
│   │   └── i16[512]
│   ├── frac
│   │   └── f32[512, 3]
│   ├── cart
│   │   └── f32[512, 3]
│   ├── incoming
│   │   └── i32[512, 32]
│   ├── incoming_pad
│   │   └── bool[512, 32]
│   ├── outgoing
│   │   └── i32[512, 20]
│   ├── outgoing_pad
│   │   └── bool[512, 20]
│   └── graph_i
│       └── i32[512]
├── edges
│   ├── to_jimage
│   │   └── i8[9872, 3]
│   ├── graph_i
│   │   └── i32[9872]
│   ├── sender
│   │   └── i32[9872]
│   └── receiver
│       └── i32[9872]
├── n_node
│   └── i32[64]
├── n_edge
│   └── i32[64]
├── padding_mask
│   └── bool[64]
└── graph_data
    ├── dataset_i
    │   └── i32[64]
    ├── abc
    │   └── f32[64, 3]
    ├── angles_rad
    │   └── f32[64, 3]
    ├── lat
    │   └── f32[64, 3, 3]
    ├── e_form
    │   └── f32[64]
    ├── bandgap
    │   └── f32[64]
    ├── e_total
    │   └── f32[64]
    ├── ehull
    │   └── f32[64]
    ├── dimensionality
    │   └── f32[64]
    ├── density
    │   └── f32[64]
    ├── space_group
    │   └── f32[64]
    ├── magmom
    │   └── f32[64]
    └── num_spec
        └── f32[64]

In [6]:
cg1 = load_file(config, 1, pad=True)
cg2 = load_file(config, 2, pad=True)
cg3 = load_file(config, 3, pad=True)
cgs = jax.tree_map(lambda x, y, z: jnp.stack([x, y, z]), cg1, cg2, cg3)
debug_structure(cgs);

arg0 >>> CrystalGraphs
├── nodes
│   ├── species
│   │   └── i16[3, 512]
│   ├── frac
│   │   └── f32[3, 512, 3]
│   ├── cart
│   │   └── f32[3, 512, 3]
│   ├── incoming
│   │   └── i32[3, 512, 32]
│   ├── incoming_pad
│   │   └── bool[3, 512, 32]
│   ├── outgoing
│   │   └── i32[3, 512, 20]
│   ├── outgoing_pad
│   │   └── bool[3, 512, 20]
│   └── graph_i
│       └── i32[3, 512]
├── edges
│   ├── to_jimage
│   │   └── i8[3, 9872, 3]
│   ├── graph_i
│   │   └── i32[3, 9872]
│   ├── sender
│   │   └── i32[3, 9872]
│   └── receiver
│       └── i32[3, 9872]
├── n_node
│   └── i32[3, 64]
├── n_edge
│   └── i32[3, 64]
├── padding_mask
│   └── bool[3, 64]
└── graph_data
    ├── dataset_i
    │   └── i32[3, 64]
    ├── abc
    │   └── f32[3, 64, 3]
    ├── angles_rad
    │   └── f32[3, 64, 3]
    ├── lat
    │   └── f32[3, 64, 3, 3]
    ├── e_form
    │   └── f32[3, 64]
    ├── bandgap
    │   └── f32[3, 64]
    ├── e_total
    │   └── f32[3, 64]
    ├── ehull
    │   └── f32[3, 64]
    ├── dimensionality
    │   └── f32[3, 64]
    ├── density
    │   └── f32[3, 64]
    ├── space_group
    │   └── f32[3, 64]
    ├── magmom
    │   └── f32[3, 64]
    └── num_spec
        └── f32[3, 64]

In [7]:
from jax.experimental import mesh_utils
from jax.sharding import Mesh, PartitionSpec as P, NamedSharding

vae = config.build_vae()
params = ckpt['state']['params']


In [8]:
from cdv.layers import Context


mesh = Mesh(mesh_utils.create_device_mesh((3,)), 'batch')
sharding = NamedSharding(mesh, P('batch'))
replicated_sharding = NamedSharding(mesh, P())

ctx = Context(training=False)

cgs = jax.device_put(cgs, sharding)
params = jax.device_put(params, replicated_sharding)

vae.apply(params, cgs, ctx=ctx)

E0815 15:03:07.244716  834254 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.
E0815 15:03:07.245164  834256 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:395: NCCL operation ncclCommInitRankConfig(&comm_handle, nranks, AsNcclUniqueId(clique_id), ranks[i].rank, &comm_config) failed: unhandled system error. Last NCCL warning(error) log entry (may be unrelated) 'Unable to load NCCL library. Multi-GPU collectives will not work.'.
E0815 15:03:07.245193  834258 pjrt_stream_executor_client.cc:2804] Execution of replica 0 failed: INTERNAL: external/xla/xla/service/gpu/nccl_api.cc:3

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 cgs = jax.device_put(cgs, sharding)                                                         │
│   11 params = jax.device_put(params, replicated_sharding)                                        │
│   12                                                                                             │
│ ❱ 13 vae.apply(params, cgs, ctx=ctx)                                                             │
│   14                                                                                             │
│                                                                                                  │
│ /home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/jax/_src/traceback_util.py:17 │
│ 9 in reraise_with_filtered_traceback                                                             │
│                                                                                                  │
│   176   def reraise_with_filtered_traceback(*args, **kwargs):                                    │
│   177 │   __tracebackhide__ = True                                                               │
│   178 │   try:                                                                                   │
│ ❱ 179 │     return fun(*args, **kwargs)                                                          │
│   180 │   except Exception as e:                                                                 │
│   181 │     mode = _filtering_mode()                                                             │
│   182 │     if _is_under_reraiser(e) or mode == "off":                                           │
│                                                                                                  │
│ /home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/flax/linen/module.py:2226 in  │
│ apply                                                                                            │
│                                                                                                  │
│   2223 │   elif method is None:                                                                  │
│   2224 │     method = self.__call__                                                              │
│   2225 │   method = _get_unbound_fn(method)                                                      │
│ ❱ 2226 │   return apply(                                                                         │
│   2227 │     method,                                                                             │
│   2228 │     self,                                                                               │
│   2229 │     mutable=mutable,                                                                    │
│                                                                                                  │
│ /home/nmiklaucic/miniconda3/envs/avid/lib/python3.12/site-packages/flax/core/scope.py:1101 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   1098 │   with bind(                                                                            │
│   1099 │     variables, rngs=rngs, mutable=mutable, flags=flags                                  │
│   1100 │   ).temporary() as root:                                                                │
│ ❱ 1101 │     y = fn(root, *args, **kwargs)                                                       │
│   1102 │   if mutable is not False:                                                              │
│   1103 │     return y, root.mutable_variables()            

In [6]:
clean = pd.read_pickle('precomputed/jarvis_dft3d_cleaned/dataframe.pkl')
clean.iloc[30746]


space_group                                                       2
formula                                                  (Nb, O, F)
e_form                                                     -2.97295
bandgap                                                       3.561
atoms             [[-0.26378997  1.36032922  3.78503205] Nb, [2....
e_total                                                    -4.95885
ehull                                                        0.0058
dimensionality                                              2D-bulk
density                                                       3.612
num_atoms                                                        20
num_spec                                                          3
magmom                                                          0.0
Name: 36725, dtype: object

In [7]:
jnp.einsum('bij,bi->bj', batch.globals.lat[batch.nodes.graph_i], batch.nodes.frac)


Array([[-0.26379   ,  1.3603295 ,  3.7850323 ],
       [ 2.1203477 ,  1.3777361 ,  0.83578944],
       [ 2.674069  ,  0.9959506 ,  6.254862  ],
       ...,
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]], dtype=float32)

In [8]:
rng = jr.key(123)

lat_inv = jax.vmap(jnp.linalg.inv)(batch.globals.lat + jnp.eye(3) * 1e-3)

def to_cart(fracs):
    return jnp.einsum('bij,bi->bj', batch.globals.lat[batch.nodes.graph_i], fracs)

def to_frac(carts):
    return jnp.einsum('bij,bi->bj', lat_inv[batch.nodes.graph_i], carts)


start_frac = jr.uniform(rng, batch.nodes.frac.shape)
start_cart = to_cart(start_frac)
start = batch.replace(nodes=batch.nodes.replace(cart=start_cart, frac=None))

In [67]:
from tqdm import tqdm, trange

from cdv.layers import Context
num_steps = 35
steps = jnp.linspace(1e-1, 1e-3, num_steps)

seq = [start]

self = model
ctx = Context(training=False)

for step_size in tqdm(steps):
    cg = seq[-1]
    z = self.encoder(cg, ctx)
    # debug_stat(z=z, eps=eps, noisy_cg=noisy_cg)
    eps_rec = self.decoder(cg, z, ctx)  
    new_cart = cg.nodes.cart + eps_rec * step_size
    new_cart = to_cart(to_frac(new_cart) % 1)
    
    new = cg.replace(nodes=cg.nodes.replace(cart=new_cart, frac=None))
    seq.append(new)

100%|██████████| 35/35 [00:14<00:00,  2.46it/s]


In [71]:
coords = jnp.stack([to_frac(s.nodes.cart) for s in seq])

# coords[:, 0]

In [70]:
gen = seq[-1]
# gen

In [89]:
from collections.abc import Sequence
import plotly.express as px
import plotly.graph_objects as go
from pymatgen.core import Structure

from cdv.databatch import CrystalGraphs
elements = config.data.metadata['elements']
jmol = pd.read_csv('https://raw.githubusercontent.com/CorySimon/JMolColors/master/jmolcolors.csv').set_index('atom')
jmol_map = {
    i: '#' + row['Hex']
    for i, row in jmol.iterrows()
}

def cg_to_struct(cg: CrystalGraphs, batch_i: int = 0):
    mask = (cg.nodes.graph_i == batch_i) & cg.padding_mask[cg.nodes.graph_i]    
    coords = np.array(cg.nodes.cart)[mask]
    species_i = np.array(cg.nodes.species)[mask]
    species = [elements[i] for i in species_i]
    lat = cg.globals.lat[batch_i]
    return Structure(lat, species, coords, coords_are_cartesian=True)

def plot_struct(structs: Structure | Sequence[Structure], last_frame='actual'):
    if isinstance(structs, Structure):
        structs = [structs]
    
    sdfs = []
    for i, struct in enumerate(structs):
        sdf = pd.DataFrame(struct.frac_coords % 1, columns=['x', 'y', 'z'])
        sdf['atom'] = struct.species
        sdf['frame'] = i
        if last_frame is not None and i == len(structs) - 1:
            sdf['frame'] = last_frame
        sdfs.append(sdf)

    sdf = pd.concat(sdfs)
    range_xyz = [-0.05, 1.05]
    return px.scatter_3d(sdf, x='x', y='y', z='z', color='atom', color_discrete_map=jmol_map, animation_frame='frame',
                         range_x=range_xyz, range_y=range_xyz, range_z=range_xyz)

struct = clean.iloc[30746]['atoms']
batch_i = 19
# plot_struct(struct)
plot_struct(cg_to_struct(batch, batch_i))

In [90]:
plot_struct([cg_to_struct(s, batch_i) for s in seq] + [cg_to_struct(batch, batch_i)])

In [88]:
clean.iloc[batch.globals.dataset_i]['space_group'].head(20)


36725      2
70987      2
25537     31
5538      38
1119       1
23748     62
21512    227
70005     62
38655     74
25547     63
62938    166
47019    123
40228     63
25437     11
68117     62
49374     12
74554    189
23776    146
45225      5
29215    225
Name: space_group, dtype: int64